<a href="https://colab.research.google.com/github/HJJunn/hands-on-machine-learning/blob/main/11_%EC%8B%AC%EC%B8%B5_%EC%8B%A0%EA%B2%BD%EB%A7%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 파이썬 ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 ≥0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"

# 텐서플로 ≥2.0 필수
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

# 공통 모듈 임포트
import numpy as np
import os

# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)

# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장:", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

He 초기화

In [ ]:
keras.layers.Dense(10, activation = "relu", kernel_initializer = "he_normal")

<Dense name=dense, built=False>

균등분포 He 초기화

In [ ]:
he_avg_init = keras.initializers.VarianceScaling(scale = 2, mode = 'fan_avg', distribution= 'uniform')
keras.layers.Dense(10, activation = "relu", kernel_initializer = he_avg_init)

<Dense name=dense_1, built=False>

수렴하지 않는 활성화 함수

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


LeakyReLU를 이용한 MNIST

In [ ]:
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)


In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape = [28,28]),
    keras.layers.Dense(300, kernel_initializer = "he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(100, kernel_initializer = "he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(10, activation = "softmax")
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.compile(loss = "sparse_categorical_crossentropy",
              optimizer = keras.optimizers.SGD(learning_rate = 1e-3),
              metrics = ['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs = 10, validation_data = (X_valid, y_valid))

Epoch 1/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.4912 - loss: 1.6183 - val_accuracy: 0.7326 - val_loss: 0.8442
Epoch 2/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.7372 - loss: 0.8036 - val_accuracy: 0.7818 - val_loss: 0.6852
Epoch 3/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.7784 - loss: 0.6747 - val_accuracy: 0.8004 - val_loss: 0.6164
Epoch 4/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.7964 - loss: 0.6125 - val_accuracy: 0.8130 - val_loss: 0.5754
Epoch 5/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8074 - loss: 0.5735 - val_accuracy: 0.8216 - val_loss: 0.5472
Epoch 6/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8147 - loss: 0.5463 - val_accuracy: 0.8260 - val_loss: 0.5263
Epoch 7/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8209 - loss: 0.5259 - val_accuracy: 0.8298 - val_loss: 0.5101
Epoch 8/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8263 - loss: 0.5099 -

SELU

SELU 활성화 함수와 LeCun초기화를 사용하면 자기정규화가 되면서 각 층의 출력이 평균과 표준편차를 보존하는 경향이 있어 gradients 소실 문제를 막아준다.(시퀀셜 CNN)

기본적으로 SELU 하이퍼파라미터(scale과 alpha)는 각 뉴런의 평균 출력이 0에 가깝고 표준 편차는 1에 가깝도록 조정됩니다(입력은 평균이 0이고 표준 편차 1로 표준화되었다고 가정합니다). 이 활성화 함수를 사용하면 1,000개의 층이 있는 심층 신경망도 모든 층에 걸쳐 거의 평균이 0이고 표준 편차를 1로 유지합니다. 이를 통해 그레이디언트 폭주와 소실 문제를 피할 수 있습니다:

SELU를 활용한 MNIST

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="selu",
                             kernel_initializer="lecun_normal"))
for layer in range(99):
    model.add(keras.layers.Dense(100, activation="selu",
                                 kernel_initializer="lecun_normal"))
model.add(keras.layers.Dense(10, activation="softmax"))

In [ ]:
model.compile(loss = "sparse_categorical_crossentropy",
              optimizer = keras.optimizers.SGD(learning_rate =1e-3),
              metrics = ["accuracy"])

입력 평균:0, 표준 편차: 1

In [ ]:
pixel_mean = X_train.mean(axis = 0, keepdims = True)
pixel_std = X_train.std(axis = 0, keepdims = True)
X_train_scaled = (X_train - pixel_mean) / pixel_std
X_valid_scaled = (X_valid - pixel_mean) / pixel_std
X_test_sclaed = (X_test - pixel_mean) / pixel_std

In [ ]:
history = model.fit(X_train_scaled, y_train, epochs = 5, validation_data = (X_valid_scaled, y_valid))

Epoch 1/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 33s 16ms/step - accuracy: 0.4566 - loss: 1.4553 - val_accuracy: 0.6514 - val_loss: 0.9117
Epoch 2/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 40s 15ms/step - accuracy: 0.6961 - loss: 0.8010 - val_accuracy: 0.7710 - val_loss: 0.6383
Epoch 3/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 39s 14ms/step - accuracy: 0.7288 - loss: 0.7300 - val_accuracy: 0.7058 - val_loss: 0.7642
Epoch 4/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 41s 15ms/step - accuracy: 0.6999 - loss: 0.7552 - val_accuracy: 0.7346 - val_loss: 0.6932
Epoch 5/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 41s 15ms/step - accuracy: 0.7332 - loss: 0.6859 - val_accuracy: 0.7530 - val_loss: 0.6229


배치 정규화

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape = [28,28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation = "elu", kernel_initializer = "he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation = "elu", kernel_initializer = "he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation = "softmax")
])

In [ ]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_4 (Flatten)                  │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 784)                 │           3,136 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_108 (Dense)                    │ (None, 300)                 │         235,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 300)                 │           1,200 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_109 (Dense)                    │ (None, 100)                 │          30,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 100)                 │             400 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_110 (Dense)                    │ (None, 10)                  │           1,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 271,346 (1.04 MB)

 Trainable params: 268,978 (1.03 MB)

 Non-trainable params: 2,368 (9.25 KB)

배치 정규화 층 4개의 파라미터 생성

첫번째 배치 정규화 층 4 * 784 = 3136개 파라미터

총 배치 정규화 파라미터 개수 3136 + 1200 + 400개

이를 2로 나누면 훈련되지 않은 전체 파라미터 개수 2368개

몇몇은 활성화 함수 이전에 배치 정규화층을 추가하는 것이 좋다고 한다.

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape = [28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, kernel_initializer = "he_normal", use_bias= True), #이전층에서 편항을 뺄 수 있다.
    keras.layers.BatchNormalization(),
    keras.layers.Activation("elu"),
    keras.layers.Dense(100, kernel_initializer = "he_normal", use_bias = True),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("elu"),
    keras.layers.Dense(10, activation = "softmax")
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.compile(loss = "sparse_categorical_crossentropy",
              optimizer = keras.optimizers.SGD(learning_rate = 1e-3),
              metrics = ["accuracy"])


In [ ]:
history = model.fit(X_train, y_train, epochs = 5, validation_data = (X_valid,y_valid))

Epoch 1/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.5796 - loss: 1.2528 - val_accuracy: 0.7892 - val_loss: 0.6280
Epoch 2/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.7735 - loss: 0.6635 - val_accuracy: 0.8154 - val_loss: 0.5402
Epoch 3/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.7995 - loss: 0.5856 - val_accuracy: 0.8284 - val_loss: 0.4998
Epoch 4/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.8117 - loss: 0.5457 - val_accuracy: 0.8364 - val_loss: 0.4751
Epoch 5/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.8195 - loss: 0.5198 - val_accuracy: 0.8422 - val_loss: 0.4580


사전 훈련된 층 재사용하기




*   X_train_A: 샌달과 셔츠(클래스 5와 6)을 제외한 모든 이미지

*  X_train_B: 샌달과 셔츠 이미지 중 처음 200개만 가진 작은 훈련 세트




In [ ]:
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [ ]:
X_train_A.shape

(43986, 28, 28)

In [ ]:
X_train_B.shape

(200, 28, 28)

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape = [28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation = "selu"))
model_A.add(keras.layers.Dense(8, activation= "softmax"))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model_A.compile(loss = "sparse_categorical_crossentropy",
              optimizer = keras.optimizers.SGD(learning_rate =1e-3),
              metrics = ["accuracy"])

In [ ]:
history = model_A.fit(X_train_A, y_train_A, epochs = 10, validation_data = (X_valid_A, y_valid_A))

Epoch 1/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.7198 - loss: 0.8784 - val_accuracy: 0.8570 - val_loss: 0.3937
Epoch 2/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.8684 - loss: 0.3747 - val_accuracy: 0.8822 - val_loss: 0.3347
Epoch 3/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.8843 - loss: 0.3277 - val_accuracy: 0.8901 - val_loss: 0.3103
Epoch 4/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.8922 - loss: 0.3053 - val_accuracy: 0.8986 - val_loss: 0.2951
Epoch 5/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.8967 - loss: 0.2908 - val_accuracy: 0.9018 - val_loss: 0.2849
Epoch 6/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.9000 - loss: 0.2803 - val_accuracy: 0.9056 - val_loss: 0.2776
Epoch 7/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.9040 - loss: 0.2720 - val_accuracy: 0.9061 - val_loss: 0.2721
Epoch 8/10
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.9073 - loss: 0.2653 

In [ ]:
model_A.save("my_model_A.h5")

In [ ]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape = [28,28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation = "selu"))
model_B.add(keras.layers.Dense(1, activation = "sigmoid"))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model_B.compile(loss = "binary_crossentropy",
                optimizer = keras.optimizers.SGD(learning_rate= 1e-3),
                metrics = ["accuracy"])

In [ ]:
history = model_B.fit(X_train_B, y_train_B, epochs = 10, validation_data= (X_valid_B, y_valid_B))

Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - accuracy: 0.4283 - loss: 0.9347 - val_accuracy: 0.7525 - val_loss: 0.5547
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.7512 - loss: 0.5266 - val_accuracy: 0.8387 - val_loss: 0.4142
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.8471 - loss: 0.3912 - val_accuracy: 0.8854 - val_loss: 0.3306
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.8810 - loss: 0.3096 - val_accuracy: 0.9199 - val_loss: 0.2751
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.9378 - loss: 0.2558 - val_accuracy: 0.9402 - val_loss: 0.2366
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.9628 - loss: 0.2190 - val_accuracy: 0.9462 - val_loss: 0.2084
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9734 - loss: 0.1923 - val_accuracy: 0.9533 - val_loss: 0.1869
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.9734 - loss: 0.1720 - val_accuracy: 0.9655 - val_loss: 0.1697


In [ ]:
model_B.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_7 (Flatten)                  │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_120 (Dense)                    │ (None, 300)                 │         235,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_121 (Dense)                    │ (None, 100)                 │          30,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_122 (Dense)                    │ (None, 50)                  │           5,050 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_123 (Dense)                    │ (None, 50)                  │           2,550 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_124 (Dense)                    │ (None, 50)                  │           2,550 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_125 (Dense)                    │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 275,803 (1.05 MB)

 Trainable params: 275,801 (1.05 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [ ]:
model_A = keras.models.load_model("my_model_A.h5")

출력층만 제외하고 모든 층 재사용

In [ ]:
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation = "sigmoid"))

model_A와 model_B는 일부 층을 공유해서 같이 영향을 받음

이를 원치 않으면 model_A를 clone

In [ ]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

재사용된 층 동결, 새로운 층에 적절한 가중치 학습

In [ ]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False
model_B_on_A.compile(loss = "binary_crossentropy",
                     optimizer = keras.optimizers.SGD(learning_rate = 1e-3),
                     metrics = ['accuracy'])

In [ ]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs = 4, validation_data = (X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True
optimizer = keras.optimizers.SGD(learning_rate = 1e-4)
model_B_on_A.compile(loss = 'binary_crossentropy',
                     optimizer = keras.optimizers.SGD(learning_rate = 1e-3),
                     metrics = ['accuracy'])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs = 16, validation_data = (X_valid_B, y_valid_B))

Epoch 1/4
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.6857 - loss: 0.5261 - val_accuracy: 0.7191 - val_loss: 0.5064
Epoch 2/4
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.7245 - loss: 0.4852 - val_accuracy: 0.7485 - val_loss: 0.4703
Epoch 3/4
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.7530 - loss: 0.4491 - val_accuracy: 0.7809 - val_loss: 0.4384
Epoch 4/4
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.7902 - loss: 0.4171 - val_accuracy: 0.8093 - val_loss: 0.4099
Epoch 1/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.8655 - loss: 0.3551 - val_accuracy: 0.9158 - val_loss: 0.2727
Epoch 2/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.9238 - loss: 0.2393 - val_accuracy: 0.9402 - val_loss: 0.2166
Epoch 3/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.9408 - loss: 0.1874 - val_accuracy: 0.9564 - val_loss: 0.1822
Epoch 4/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.9700 - loss: 0.1549 - val_accuracy: 0.9665 - val_loss: 0.1579
Epoc

In [ ]:
model_B.evaluate(X_test_B, y_test_B)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9803 - loss: 0.1370


[0.1396910697221756, 0.9779999852180481]

In [ ]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9934 - loss: 0.0595


[0.06383442878723145, 0.9925000071525574]

In [ ]:
(100-97.79)/(100-99.25)

2.9466666666666583

고속 옵티마이저


In [ ]:
#모멘텀
optimizer = keras.optimizers.SGD(learning_rate = 0.001, momentum = 0.9)
#네스테로프 가속 경사 (nag)
optimizer = keras.optimizers.SGD(learning_rate = 0.001, momentum = 0.9, nesterov = True)
#AdaGrad
optimizer = keras.optimizers.Adagrad(learning_rate = 0.001)
#RMSProp
optimizer = keras.optimizers.RMSprop(learning_rate = 0.001, rho = 0.9)
#Adam
optimizer = keras.optimizers.Adam(learning_rate = 0.001, beta_1 = 0.9, beta_2 = 0.999)
#Adamax
optimizer = keras.optimizers.Adamax(learning_rate = 0.001, beta_1 = 0.9, beta_2 = 0.999)
#Nadam
optimizer = keras.optimizers.Nadam(learning_rate = 0.001, beta_1 = 0.9, beta_2 = 0.999)

학습률 스케쥴링